In [1]:
from foreduce.data.data import GraphDataset

trainset = GraphDataset.load(f'./data/666_train.pt')

/home/apluska/foreduce/foreduce/data/data.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  max_arity, data, labels = torch.load(path)


In [27]:
import torch

x = torch.tensor([1.0, 0.0, 1.0])
y = torch.tensor([0.5, 0.0, 0.5])

alpha = 1.0

torch.nn.functional.cross_entropy(x, y, label_smoothing=alpha)

tensor(1.1953)

In [2]:
graph, labels = trainset[0]

In [3]:
from torch_geometric.loader import DataLoader

loader  = DataLoader(trainset, batch_size=1, shuffle=True)

In [5]:
from foreduce.transformer.model import GraphModel
from foreduce.data.data import _type_mapping


model = GraphModel(len(_type_mapping), trainset.max_arity, 6, 64)

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
b.to(torch.float).reshape(-1) / torch.sum(b)

tensor([0.0000, 0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])

In [16]:
b.shape[1]

11

In [17]:
import torch

torch.nn.functional.cross_entropy(model(a), b.to(torch.float).reshape(-1) / torch.sum(b)) / b.shape[1]

tensor(0.2224, grad_fn=<DivBackward0>)

In [67]:
from foreduce.vampire.vampire import VampireInteractive
import os
import torch

VAMPIRE = os.getenv("VAMPIRE")


interactive = VampireInteractive(VAMPIRE, f'./problems/SYN/SYN986+1.005.p')
interactive.__enter__()
symbols = interactive.problem.function_symbols() | interactive.problem.predicate_symbols()
permutation = torch.randperm(32)
name = {s: permutation[i] + 1 for i, s in enumerate(symbols)}

In [68]:
interactive.problem

goal_0 | ~r(zero, X4, X5) | ~r(zero, X3, X4) | ~r(zero, X2, X3) | ~r(zero, X1, X2) | ~r(zero, X0, X1) | ~r(zero, zero, X0)
r(X6, zero, succ(X6))
r(X7, succ(X8), X10) | ~r(X9, X8, X10) | ~r(X7, X8, X9)
~goal_0

In [69]:
graphs = []
datalist = []

In [70]:
from torch_geometric.utils.convert import from_networkx
from torch_geometric.data import Batch

from foreduce.data.data import _type_mapping


In [71]:
from foreduce.transformer.model import Model

model = Model.load_from_checkpoint(f'./models/test.ckpt')

In [72]:
active = torch.zeros(1024, dtype=torch.bool)

In [73]:
import sys
sys.setrecursionlimit(10000)

In [138]:
graphs = interactive.problem.extend(graphs, len(graphs), depth=8)
for graph in graphs[len(datalist):]:
    data = from_networkx(graph)
    data.type = torch.tensor([_type_mapping[t] for t in data.type], dtype=torch.int)
    data.name = torch.tensor([name[s] if s is not None else 0 for s in data.name], dtype=torch.int)
    data.arity = torch.tensor([min(4 + 1, a + 1) if a is not None else 0 for a in data.arity], dtype=torch.int)
    data.pos = torch.tensor([min(4 + 1, a + 1) if a is not None else 0 for a in data.pos], dtype=torch.int)
    datalist.append(data)
score = model(Batch.from_data_list(datalist))
given = torch.where(active[:len(graphs)], -10000, score).argmax().item()
active[given] = True
interactive.step(given)
interactive

 0: goal_0 | ~r(zero, X4, X5) | ~r(zero, X3, X4) | ~r(zero, X2, X3) | ~r(zero, X1, X2) | ~r(zero, X0, X1) | ~r(zero, zero, X0)
 1: r(X6, zero, succ(X6))
 2: r(X7, succ(X8), X10) | ~r(X9, X8, X10) | ~r(X7, X8, X9)
 3: ~goal_0
 4: ~r(zero, X0, X1) | ~r(zero, X2, X0) | ~r(zero, X3, X2) | ~r(zero, X4, X3) | ~r(zero, succ(zero), X4) | goal_0
 5: r(X0, succ(zero), succ(X1)) | ~r(X0, zero, X1)
 6: ~r(zero, zero, X0) | ~r(zero, X1, X2) | ~r(zero, X3, X1) | ~r(zero, succ(X0), X3) | ~r(zero, X2, X4) | goal_0
 7: ~r(X5, zero, X6) | r(X7, succ(succ(zero)), succ(X6)) | ~r(X7, succ(zero), X5)
 8: ~r(zero, zero, X0) | ~r(zero, X1, X2) | ~r(zero, X3, X1) | ~r(zero, succ(X0), X3) | ~r(zero, X2, X4)
 9: r(X0, succ(succ(zero)), succ(X1)) | ~r(succ(X2), zero, X1) | ~r(X0, zero, X2)
 10: r(X0, succ(succ(zero)), succ(succ(succ(X1)))) | ~r(X0, zero, X1)
 11: ~r(X0, zero, X1) | r(X2, succ(succ(succ(zero))), succ(succ(succ(X1)))) | ~r(X2, succ(succ(zero)), X0)
 12: r(X0, succ(succ(succ(zero))), succ(succ(succ(

In [57]:
sum(len(g) for g in graphs)

1272

In [56]:
print(interactive.proof)

% Running in auto input_syntax mode. Trying TPTP
[SA] new: 1. goal_0 | ~r(zero,X4,X5) | ~r(zero,X3,X4) | ~r(zero,X2,X3) | ~r(zero,X1,X2) | ~r(zero,X0,X1) | ~r(zero,zero,X0) [input]
[SA] new: 2. r(X6,zero,succ(X6)) [input]
[SA] new: 3. r(X7,succ(X8),X10) | ~r(X9,X8,X10) | ~r(X7,X8,X9) [input]
[SA] new: 4. ~goal_0 [input]
[SA] new: 5. r(X0,succ(zero),succ(X1)) | ~r(X0,zero,X1) [resolution 3,2]
[SA] new: 6. ~r(X0,zero,X1) | r(X2,succ(succ(zero)),succ(X1)) | ~r(X2,succ(zero),X0) [resolution 5,3]
[SA] new: 7. r(X0,succ(succ(zero)),succ(X1)) | ~r(succ(X2),zero,X1) | ~r(X0,zero,X2) [resolution 6,5]
[SA] new: 8. r(X0,succ(succ(zero)),succ(succ(succ(X1)))) | ~r(X0,zero,X1) [resolution 7,2]
[SA] new: 9. ~r(X0,zero,X1) | r(X2,succ(succ(succ(zero))),succ(succ(succ(X1)))) | ~r(X2,succ(succ(zero)),X0) [resolution 8,3]
[SA] new: 10. r(X0,succ(succ(succ(zero))),succ(succ(succ(X1)))) | ~r(succ(succ(succ(X2))),zero,X1) | ~r(X0,zero,X2) [resolution 9,8]
[SA] new: 11. r(X0,succ(succ(succ(zero))),succ(succ

In [145]:
graphs = []
datalist = []
active = torch.zeros(1024, dtype=torch.bool)
with VampireInteractive(VAMPIRE, f'./problems/GRP/GRP001-2.p') as interactive:
    symbols = interactive.problem.function_symbols() | interactive.problem.predicate_symbols()
    permutation = torch.randperm(32)
    name = {s: permutation[i] + 1 for i, s in enumerate(symbols)}
    while not interactive.finished and interactive.step_count < 128 and (
        sum(len(g) for g in graphs) < 2**20
    ) and (len(graphs) == 0 or not active[:len(graphs)].all()):
        graphs = interactive.problem.extend(graphs, len(graphs), depth=4)
        for graph in graphs[len(datalist):]:
            data = from_networkx(graph)
            data.type = torch.tensor([_type_mapping[t] for t in data.type], dtype=torch.int)
            data.name = torch.tensor([name[s] if s is not None else 0 for s in data.name], dtype=torch.int)
            data.arity = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.arity], dtype=torch.int)
            data.pos = torch.tensor([min(8 + 1, a + 1) if a is not None else 0 for a in data.pos], dtype=torch.int)
            datalist.append(data)
        score = model(Batch.from_data_list(datalist))
        given = torch.where(active[:len(graphs)], -10000, score).argmax().item()
        active[given] = True
        interactive.step(given)

RuntimeError: The size of tensor a (1024) must match the size of tensor b (1108) at non-singleton dimension 0

In [146]:
print(interactive.proof)

% Running in auto input_syntax mode. Trying TPTP
[SA] new: 1. multiply(X0,identity) = X0 [input]
[SA] new: 2. identity = multiply(X1,inverse(X1)) [input]
[SA] new: 3. identity = multiply(X2,X2) [input]
[SA] new: 4. multiply(a,b) = c [input]
[SA] new: 5. goal_0 | c != multiply(b,a) [input]
[SA] new: 6. multiply(identity,X3) = X3 [input]
[SA] new: 7. identity = multiply(inverse(X4),X4) [input]
[SA] new: 8. multiply(multiply(X5,X6),X7) = multiply(X5,multiply(X6,X7)) [input]
[SA] new: 9. ~goal_0 [input]
[SA] new: 10. identity = inverse(identity) [superposition 1,7]
[SA] new: 11. identity = inverse(identity) [superposition 7,1]
[SA] new: 12. identity = multiply(identity,identity) [superposition 7,10]
[SA] new: 13. multiply(inverse(X0),multiply(X0,X1)) = multiply(identity,X1) [superposition 8,7]
[SA] new: 14. multiply(identity,X3) = multiply(X2,multiply(X2,X3)) [superposition 8,3]
[SA] new: 15. multiply(X4,X5) = multiply(identity,multiply(X4,X5)) [superposition 8,6]
[SA] new: 16. multiply(mu